In [4]:
import numpy as np
import pandas as pd

In [5]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [6]:
train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

In [4]:
train.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,6,1,0,0,0,0,7,0,...,0,0,0,0,0,0,2,0,0,Class_6
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,0,0,0,0,0,1,0,3,0,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,0,0,7,0,1,5,2,2,0,1,...,0,4,0,2,2,0,4,3,0,Class_8
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [5]:
import missingno as msno
# msno.matrix(df=train, figsize=(20,14), color=(0.5,0,0))

In [6]:
train_x = train.drop(['target'], axis=1)
train_y = train['target']

test_x = test.copy()

In [7]:
train_x.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,0,0,6,1,0,0,0,0,7,0,...,3,0,0,0,0,0,0,2,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,3,0,0,...,8,0,0,0,0,1,0,0,0,0
3,0,0,7,0,1,5,2,2,0,1,...,0,0,4,0,2,2,0,4,3,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_y.head()

0    Class_6
1    Class_6
2    Class_2
3    Class_8
4    Class_2
Name: target, dtype: object

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y)
train_y = pd.Series(train_y)

In [10]:
train_y.head()

0    5
1    5
2    1
3    7
4    1
dtype: int64

In [11]:
#import lightgbm as lgb
import xgboost as xgb
#import optuna.integration.lightgbm as lgb
from sklearn.metrics import log_loss

In [25]:
from sklearn.model_selection import train_test_split
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True, random_state=42, stratify=train_y) 

dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)

xgb_params = {
     # 多値分類問題
     'objective': 'multi:softprob',
     # クラス数
     'num_class': 3,
     # 学習用の指標 (Multiclass logloss)
     'eval_metric': 'mlogloss',
}

params = {'objective': 'multiclass',
                  #'random_state': 10,
                  'metric': 'multi_logloss',
             'num_class': 9,
             'force_row_wise' :True}

params2 = {'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 9,
 'force_row_wise': True,
 'feature_pre_filter': False,
 'lambda_l1': 9.743033661523311,
 'lambda_l2': 2.709400532224499e-07,
 'num_leaves': 33,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 100,
 'num_iterations': 50,
 'early_stopping_round': 5}

evals = [(dtrain, 'train'), (dvalid, 'eval')]
evals_result = {}


In [26]:
# train
# モデルの作成および学習データを与えての学習
model = xgb.train(xgb_params,
                    dtrain,
                    num_boost_round=1000,
                    early_stopping_rounds=10,
                    evals=evals,
                    evals_result=evals_result
                    )

XGBoostError: [23:56:07] /Users/travis/build/dmlc/xgboost/src/objective/multiclass_obj.cu:120: SoftmaxMultiClassObj: label must be in [0, num_class).
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000011fb6fde4 dmlc::LogMessageFatal::~LogMessageFatal() + 116
  [bt] (1) 2   libxgboost.dylib                    0x000000011fc68d5d xgboost::obj::SoftmaxMultiClassObj::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*) + 1053
  [bt] (2) 3   libxgboost.dylib                    0x000000011fc21cab xgboost::LearnerImpl::UpdateOneIter(int, std::__1::shared_ptr<xgboost::DMatrix>) + 667
  [bt] (3) 4   libxgboost.dylib                    0x000000011fb66a49 XGBoosterUpdateOneIter + 137
  [bt] (4) 5   libffi.7.dylib                      0x000000010a009ead ffi_call_unix64 + 85
  [bt] (5) 6   ???                                 0x00007ffee67d83a0 0x0 + 140732765406112



In [18]:
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

logloss: 1.7482


In [19]:
pred = model.predict(test_x)
print(len(pred))

100000


In [20]:
model.params

{'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 9,
 'force_row_wise': True,
 'feature_pre_filter': False,
 'lambda_l1': 9.743033661523311,
 'lambda_l2': 2.709400532224499e-07,
 'num_leaves': 33,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 100,
 'num_iterations': 50,
 'early_stopping_round': 5}

In [21]:
submission = pd.DataFrame({'id': list(range(200000,300001))})
pred = pd.DataFrame(pred,columns =["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"])
submission = pd.merge(submission,pred,left_index=True, right_index=True)
submission.to_csv('./sample_submission.csv', index=False)
print('csv output')

csv output
